In [90]:
import os
import requests
from bs4 import BeautifulSoup
from markdownify import markdownify as md

In [91]:
def extract_text_from_url(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove tags de script e estilo
        for script_or_style in soup(["script", "style"]):
            script_or_style.decompose()
        
        # Extrai o texto com um separador de espaço
        texto = soup.get_text(separator=' ')
        
        # Limpeza do texto
        linhas = (line.strip() for line in texto.splitlines())
        partes = (frase.strip() for linha in linhas for frase in linha.split("  "))
        texto_limpo = '\n'.join(linha for linha in linhas if linha)
        
        return texto_limpo
    else:
        print(f"Erro: {response.status_code}")
        return None



In [92]:
# Configurações do Azure Translator
subscription_key = 'DPo8ARgbwoI0xJvlmL7zUmwkCEE3nZ1qCyMQLDmSpIAVUlXzvAgLJQQJ99AKACHYHv6XJ3w3AAAbACOGz4oi'
endpoint = 'https://api.cognitive.microsofttranslator.com'
location = 'eastus2' 


In [93]:
def traduzir_texto(texto, subscription_key, endpoint, location, to_language='pt'):
    path = '/translate?api-version=3.0'
    params = f'&to={to_language}'
    constructed_url = endpoint + path + params

    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key,
        'Ocp-Apim-Subscription-Region': location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(os.urandom(16))
    }

    body = [{
        'text': texto
    }]

    try:
        response = requests.post(constructed_url, headers=headers, json=body)
        response.raise_for_status()
        resultados = response.json()
        traducao = ''.join([item['translations'][0]['text'] for item in resultados])
        return traducao
    except requests.exceptions.RequestException as e:
        print(f"Erro na tradução: {e}")
        return None

In [94]:
def salvar_em_markdown(texto, nome_arquivo='traducao.md'):
    markdown_text = md(texto)
    with open(nome_arquivo, 'w', encoding='utf-8') as f:
        f.write(markdown_text)
    print(f"Arquivo salvo como {nome_arquivo}")

In [96]:
url = 'https://dev.to/educative/data-science-in-5-minutes-what-is-data-cleaning-16of'
texto_extraido = extract_text_from_url(url)

if texto_extraido:
    texto_traduzido = traduzir_texto(texto_extraido, subscription_key, endpoint, location, to_language='pt')
    if texto_traduzido:
        salvar_em_markdown(texto_traduzido, 'traducao.md')

Arquivo salvo como traducao.md
